In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# cd /content/drive/MyDrive/OML---mini-project/src
%cd /content/drive/MyDrive/cours/MA/MA4 (3)/OptML/project/OML---mini-project-main/src

/content/drive/MyDrive/cours/MA/MA4 (3)/OptML/project/OML---mini-project-main/src


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cach

In [2]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.optim.lr_scheduler as schedulers
from utils import retrieve_setup, retrieve_training_params, training_loop

## CIFAR exploration

In [ ]:
model_name = "MobileNetV3Small"
dataset_name = "CIFAR10"
n_epochs = 8


lr_pos = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
lr_pairs = []

for i in range(len(lr_pos)):
  for j in range(i+1, len(lr_pos)):
    lr_pairs.append((lr_pos[i], lr_pos[j]))

print(lr_pairs)

[(0.01, 0.001), (0.01, 0.0001), (0.01, 1e-05), (0.01, 1e-06), (0.001, 0.0001), (0.001, 1e-05), (0.001, 1e-06), (0.0001, 1e-05), (0.0001, 1e-06), (1e-05, 1e-06)]


In [ ]:
for pair in lr_pairs:
  model, dataset = retrieve_setup(model_name, dataset_name)
  optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

  # change lr from default to lr_max
  for g in optimizer.param_groups:
      g["lr"] = pair[0]

  # implement linear increasing scheduler
  total_iters = int(len(dataset["train"])/batch_size*n_epochs)
  start_factor = pair[1]/pair[0]

  scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
      optimizer,
      start_factor=start_factor,
      end_factor=1,
      total_iters=total_iters
  )

  print(f'---training with lr_max={pair[0]} and lr_min={pair[1]}---')
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  _, metrics = training_loop(
      model=model,
      dataset=dataset,
      scheduler=scheduler,
      optimizer=optimizer,
      loss_fn=loss_fn,
      n_epochs=n_epochs,
      batch_size=batch_size,
      train_strategy=("iter", 25),
      test_strategy=("iter", 25),
      scheduler_strategy="iter",
      file_name=f"/{dataset_name}_{scheduler_name}_{pair[0]}_{pair[1]}",
      device=device,
  )



Files already downloaded and verified
Files already downloaded and verified
---training with lr_max=0.01 and lr_min=0.001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---training with lr_max=0.01 and lr_min=0.0001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---training with lr_max=0.01 and lr_min=1e-05---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---training with lr_max=0.01 and lr_min=1e-06---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---training with lr_max=0.001 and lr_min=0.0001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already 

## FashionMNIST exploration

In [ ]:
model_name = "MobileNetV3Small"
dataset_name = "FashionMNIST"
n_epochs = 8


lr_pos = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
lr_pairs = []

for i in range(len(lr_pos)):
  for j in range(i+1, len(lr_pos)):
    lr_pairs.append((lr_pos[i], lr_pos[j]))

print(lr_pairs)

[(0.01, 0.001), (0.01, 0.0001), (0.01, 1e-05), (0.01, 1e-06), (0.001, 0.0001), (0.001, 1e-05), (0.001, 1e-06), (0.0001, 1e-05), (0.0001, 1e-06), (1e-05, 1e-06)]


In [ ]:
for pair in lr_pairs:
  model, dataset = retrieve_setup(model_name, dataset_name)
  optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

  # change lr from default to lr_max
  for g in optimizer.param_groups:
      g["lr"] = pair[0]

  # implement linear increasing scheduler
  total_iters = int(len(dataset["train"])/batch_size*n_epochs)
  start_factor = pair[1]/pair[0]

  scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
      optimizer,
      start_factor=start_factor,
      end_factor=1,
      total_iters=total_iters
  )

  print(f'---training with lr_max={pair[0]} and lr_min={pair[1]}---')
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  _, metrics = training_loop(
      model=model,
      dataset=dataset,
      scheduler=scheduler,
      optimizer=optimizer,
      loss_fn=loss_fn,
      n_epochs=n_epochs,
      batch_size=batch_size,
      train_strategy=("iter", 25),
      test_strategy=("iter", 25),
      scheduler_strategy="iter",
      file_name=f"/{dataset_name}_{scheduler_name}_{pair[0]}_{pair[1]}",
      device=device,
  )

---training with lr_max=0.01 and lr_min=0.001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.01 and lr_min=0.0001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.01 and lr_min=1e-05---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.01 and lr_min=1e-06---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.001 and lr_min=0.0001---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.001 and lr_min=1e-05---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.001 and lr_min=1e-06---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.0001 and lr_min=1e-05---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---training with lr_max=0.0001 and lr_min=1e-06---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 

## batch size exploration


In [ ]:
model_name = "MobileNetV3Small"
dataset_name = "CIFAR10"
n_epochs = 8
lr_max = 1e-2
lr_min = 1e-4

for batch_s in [512, 1024, 2048]:



  model, dataset = retrieve_setup(model_name, dataset_name)
  optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

  # change lr from default to lr_max
  for g in optimizer.param_groups:
      g["lr"] = lr_max

  # implement linear increasing scheduler
  total_iters = int(len(dataset["train"])/batch_s*n_epochs)
  start_factor = lr_min/lr_max

  scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
      optimizer,
      start_factor=start_factor,
      end_factor=1,
      total_iters=total_iters
  )

  print(f'---testing with batch size {batch_s}---')
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  _, metrics = training_loop(
      model=model,
      dataset=dataset,
      scheduler=scheduler,
      optimizer=optimizer,
      loss_fn=loss_fn,
      n_epochs=n_epochs,
      batch_size=batch_s,
      train_strategy=("iter", 25),
      test_strategy=("iter", 25),
      scheduler_strategy="iter",
      file_name=f"/{dataset_name}_{scheduler_name}_batchs_{batch_s}_lr_4_2",
      device=device,
  )

Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 512---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 1024---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 2048---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8


## test of both dataset with bigger batch and 3 datapoints per epoch

lr_max = 5e-3

lr_min = 1e-5

In [4]:
model_name = "MobileNetV3Small"
datasets = ["CIFAR10", "FashionMNIST"]

lr_max = 5e-3
lr_min = 1e-5
n_epochs = 8

for ds in datasets:
  for batch_s in [512, 1024, 2048, 4096]:
      model, dataset = retrieve_setup(model_name, ds)
      optimizer, _, loss_fn, _, _ = retrieve_training_params(model, ds, "parameters.yml")

      # change lr from default to lr_max
      for g in optimizer.param_groups:
          g["lr"] = lr_max

      # implement linear increasing scheduler
      total_iters = int(len(dataset["train"])/batch_s*n_epochs)
      start_factor = lr_min/lr_max

      eval_iter = int((total_iters-1)/(3*n_epochs))


      scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
          optimizer,
          start_factor=start_factor,
          end_factor=1,
          total_iters=total_iters
      )

      print(f'---testing with batch size {batch_s}---')
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      _, metrics = training_loop(
          model=model,
          dataset=dataset,
          scheduler=scheduler,
          optimizer=optimizer,
          loss_fn=loss_fn,
          n_epochs=n_epochs,
          batch_size=batch_s,
          train_strategy=("iter", eval_iter),
          test_strategy=("iter", eval_iter),
          scheduler_strategy="iter",
          file_name=f"/{ds}_{scheduler_name}_3P_b{batch_s}_lrmax5em5",
          device=device,
      )





Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 512---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 1024---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 2048---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Files already downloaded and verified
Files already downloaded and verified
---testing with batch size 4096---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---testing with batch size 512---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---testing with batch size 1024---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---testing with batch size 2048---
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
---te

## single lr code

In [ ]:
# retrieves
model, dataset = retrieve_setup(model_name, dataset_name)
optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

# change lr from default to lr_max
for g in optimizer.param_groups:
    g["lr"] = lr_max

# implement linear increasing scheduler
total_iters = int(len(dataset["train"])/batch_size*n_epochs)
start_factor = lr_min/lr_max

scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
    optimizer,
    start_factor=start_factor,
    end_factor=1,
    total_iters=total_iters
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# run training loop

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, metrics = training_loop(
    model=model,
    dataset=dataset,
    scheduler=scheduler,
    optimizer=optimizer,
    loss_fn=loss_fn,
    n_epochs=n_epochs,
    batch_size=batch_size,
    train_strategy=("iter", 25),
    test_strategy=("iter", 25),
    scheduler_strategy="iter",
    file_name=f"/{dataset_name}_{scheduler_name}",
    device=device,
)